# Importing Library


In [ ]:
#importing required LIBRARY
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math


#importing LIBRARIES for LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM
import plotly.graph_objects as go
import tensorflow as tf

%matplotlib inline


Creating Required Datset

In [ ]:
#creating required dataset by appending data of past 10 years
df_nifty50 = pd.read_csv('nifty50_2011.csv')
LSTM_nifty50=df_nifty50['Close']
df_nifty50 = pd.read_csv('nifty50_2012.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2013.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2014.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2015.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2016.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2017.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2018.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2019.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2020.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)
df_nifty50 = pd.read_csv('nifty50_2021.csv')
LSTM_closing=df_nifty50['Close']
LSTM_nifty50 = LSTM_nifty50.append(LSTM_closing, ignore_index = True)




Showing plot for given data

In [ ]:
#plotting the trend line for past 10 years of Nifty Index
plt.figure(figsize = (10,5),dpi = 200)
LSTM_nifty50.plot.line()
plt.tight_layout()

Using MinMax scaler for tranforming data

In [ ]:
df2=LSTM_nifty50

#using SKLEARN tranforming our data by using min max scaler function
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1_new=scaler.fit_transform(np.array(df2).reshape(-1,1))

Creating training and testing data

In [ ]:
#creating training and testing data by splitting in 80% and 20%

training_data,testing_data=df1_new[0:int(len(df1_new)*0.80),:],df1_new[int(len(df1_new)*0.80):len(df1_new),:1]

Creating dataset in the required form for LSTM

In [ ]:

# converting the dataframe values into a matrix form of data
def dataset_new(dataset, time_step=1):
    Xinput = []
    Yinput = []
    range_len = len(dataset)-time_step
    for i in range(range_len):
        j = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        Xinput.append(j)
        Yinput.append(dataset[i + time_step, 0])
    return np.array(Xinput), np.array(Yinput) 

Splitting dataset in X and y format using dataset_new funciton

In [ ]:
#assigning value for time step
time_step = 5

#splitting the dataset in X and y form by using dataset_new function

X_train, y_train = dataset_new(training_data, time_step)
X_test, ytest = dataset_new(testing_data, time_step)

Reshaping the data in required form for LSTM

In [ ]:
xs1 = X_train.shape[0]
xs2 = X_train.shape[1]
xt1 = X_test.shape[0]
xt2 = X_test.shape[1]


#reshaping it into the format required by the LSTM model
X_train =X_train.reshape(xs1,xs2, 1)
X_test = X_test.reshape(xt1,xt2, 1)

Creating model

In [ ]:
#using sequential model for our LSTM

model=Sequential()

#input layer
model.add(LSTM(80,return_sequences=True,input_shape=(5,1)))

#hidden layer 1
model.add(LSTM(80,return_sequences=True))
model.add(LSTM(80,return_sequences=True))
model.add(LSTM(80,return_sequences=True))


#hidden layer 2
model.add(LSTM(80))



#output layer
model.add(Dense(1))



#compling all layer and use MSE as loss function
model.compile(loss='mean_squared_error',optimizer='adam')

Using earlystopping

In [ ]:
#defining early stopping 
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

Model summary

In [ ]:
#printing summary of LSTM model
model.summary()

Fitting the LSTM model

In [ ]:
#fitting the model by using early stopping in callbacks and using X_testing as validation set

model_fit=model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=200,batch_size=64,verbose=1)

Predicitng value by using Model


In [ ]:
### Predicting training and testing dataset by model generated
training_predict=model.predict(X_train)
testing_predict=model.predict(X_test)

In [ ]:
#defining function for returning predicted and actual value for any dataset fed to it
def predicting_value(filename):
    df5 = pd.read_csv(filename)
    df55 = df5['Close']
    df5_new=scaler.transform(np.array(df55).reshape(-1,1))
    
    testing_data=df5_new[0:len(df5_new),:1]
    
    time_step = 5
    X_test, ytest = dataset_new(testing_data, time_step)
    xt1 = X_test.shape[0]
    xt2 = X_test.shape[1]
    X_test = X_test.reshape(xt1,xt2, 1)
    tyu = model.predict(X_test)
    tyu_ = scaler.inverse_transform(tyu)
    tyu1 = tyu_.tolist()
    tyu2 = np.arange(len(tyu1))

    return df55[:-5],tyu1,tyu2

    

In [ ]:
#filename for which we are going to test the model
filename = 'nifty50_test_2021.csv'

#asssigning new variables to predicting_value function 
df55y = predicting_value(filename)[0]
tyu1 = predicting_value(filename)[1]
tyu2 = predicting_value(filename)[2];


Plotting prediction vs actual for any file with 'filename'

In [ ]:
#using interactive plot for plotting predicted value and actual value
import plotly.graph_objects as go

ii = []
for i in range(0,len(df55y)):
    tyunew = tyu1[i][0]
    i += 1
    ii.append(tyunew)

fig = go.Figure()

fig11=   go.Scatter(
        x=tyu2,
        y=ii,
        name = "prediction"
    )

fig22  =  go.Scatter(
        x=tyu2,
        y=df55y,
        name = "actual"
    )

data1 = [fig11,fig22]

#style layout 
layout1 = go.Layout(
    title="Nifty index Actual vs. Predicted",
    xaxis=dict(
        title="Days"
    ),
    yaxis=dict(
        title="Nifty index"
    ) ) 

fig = go.Figure(data=data1, layout=layout1)
fig.show()




Transforming data to actual value

In [ ]:
#assigning new variables for reshaping
ys1 =y_train.shape[0]
yt1 = ytest.shape[0]
y_train=y_train.reshape(ys1,1)
y_test=ytest.reshape(yt1,1)

# Using scaler function, transforming the entrie data back to original form
train_pred=scaler.inverse_transform(training_predict)
test_pred=scaler.inverse_transform(testing_predict)
y_train_act=scaler.inverse_transform(y_train)
y_test_act=scaler.inverse_transform(y_test)

Generating Metrics

In [ ]:
#importing the required LIBRARY for generating metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error


#defining function for Mean absolute precentage error
def MAPE(y_true,y_pred):
    mape=np.mean(np.abs(y_true-y_pred)/y_true)*100
    return mape


#calculating root mean squared error for train and test data
RMSE1 = (math.sqrt(mean_squared_error(y_train_act,train_pred)))
RMSE2 = (math.sqrt(mean_squared_error(y_test_act,test_pred)))

#calculating mean absolute error for train and test data
MAE1 = (mean_absolute_error(y_train_act,train_pred))
MAE2 = (mean_absolute_error(y_test_act,test_pred))

#calculating mean absolute precentage error for train and test data
MAPE1 = (MAPE(y_train_act,train_pred))
MAPE2 = (MAPE(y_test_act,test_pred))


# import module for tabulating above data
from tabulate import tabulate
  
# assign data
mydata = [["RMSE_training", RMSE1], ["RMSE_test", RMSE2],  ["MAE_train", MAE1],  ["MAE_test", MAE2], ["MAPE_train%", MAPE1], 
          ["MAPE_test%", MAPE2]]
  
# create header for values and metrics

head = ["Metric", "Value"]

# display table
print(tabulate(mydata, headers=head, tablefmt="grid"))


Plotting Train loss and Test loss for epochs run

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig1  = go.Scatter(
        x=np.arange(len(model_fit.history['loss'])),
        y=(model_fit.history['loss']),
        name = "train_loss"
    )

fig2 =  go.Scatter(
        x=np.arange(len(model_fit.history['loss'])),
        y=(model_fit.history['val_loss']),
        name = "test_loss"
    )

data = [fig1,fig2]
#style layout 
layout = go.Layout(
    title="LOSS vs epoch",
    xaxis=dict(
        title="epoch"
    ),
    yaxis=dict(
        title="loss"
    ) ) 

fig = go.Figure(data=data, layout=layout)
fig.show()
